# Beam center finder

In SANS experiments, it is essential to find the center of the scattering pattern in order to allow symmetric summation of the scattering intensity around the beam (i.e. computing a one-dimensional $I(Q)$).
As detector panels can move, the center of the beam will not always be located at the same place on the detector panel from one experiment to the next.

Here we describe the beam center finding algorithm,
which uses a combination of a center-of-mass calculation and an iterative refinement on a computed scattering cross-section to find the center of the scattering pattern.

In [ ]:
import scipp as sc
from scipp.constants import g
from ess import loki, sans
from ess.sans import beam_center_finder as bcf
from ess.logging import configure_workflow
import plopp as pp

In [ ]:
logger = configure_workflow('sans_beam_center_finder', filename='sans.log')

We begin by setting some parameters relevant to the current sample.

In [ ]:
# Include effects of gravity?
gravity = True

# Wavelength binning
wavelength_bins = sc.linspace(
    dim='wavelength', start=2.0, stop=16.0, num=141, unit='angstrom'
)

# Define Q binning
q_bins = sc.linspace('Q', 0.02, 0.3, 71, unit='1/angstrom')

# Define coordinate transformation graph
graph = sans.conversions.sans_elastic(gravity=gravity)

Next we load the data files for the sample and direct runs:

In [ ]:
sample = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063114.hdf5'))
direct = loki.io.load_sans2d(filename=loki.data.get_path('SANS2D00063091.hdf5'))
# Add X, Y coordinates
sample.coords['x'] = sample.coords['position'].fields.x
sample.coords['y'] = sample.coords['position'].fields.y
# Add gravity coordinate
sample.coords["gravity"] = sc.vector(value=[0, -1, 0]) * g

### Masking bad pixels

We create a quick image of the data (summing along the `tof` dimension) to inspect its contents.
We see a diffuse scattering pattern, centered around a dark patch with an arm to the north-east; this is the sample holder.
It is clear that the sample and the beam are not in the center of the panel, which is marked by the red dot

In [ ]:
image = sample.sum('tof').copy().hist(y=120, x=128)
p = image.plot(norm='log', aspect='equal')
p.ax.plot(0, 0, 'o', color='red', ms=5)
p

To avoid skew in future comparisons of integrated intensities between the different regions of the detector panel,
we mask out the sample holder, using a low-counts threshold.
This also masks out the edges of the panel, which show visible artifacts.
We also mask out a region in the bottom right corner where a group of hot pixels is apparent.
Finally, there is a single hot pixel in the detector on the right edge of the panel with counts in excess of 1000,
which we also remove.

In [ ]:
summed = sample.sum('tof')

low_counts = summed.data < sc.scalar(70, unit='counts')
high_counts = summed.data > sc.scalar(1000, unit='counts')
lower_right = (sample.coords['x'] > sc.scalar(0.35, unit='m')) & (
    sample.coords['y'] < sc.scalar(-0.4, unit='m')
)

sample.masks['low_counts'] = low_counts
sample.masks['high_counts'] = high_counts
sample.masks['lower_right'] = lower_right

We look at the image again, to verify we have masked the desired regions.

In [ ]:
image = sample.sum('tof').copy().hist(y=120, x=128)
p = image.plot(norm='log', aspect='equal')
p.ax.plot(0, 0, 'o', color='red', ms=5)
p

## Description of the procedure

The procedure to determine the precise location of the beam center is the following:

1. obtain an initial guess by computing the center-of-mass of the pixels, weighted by the counts on each pixel
2. from that initial guess, divide the panel into 4 quadrants
3. compute $I(Q)$ inside each quadrant and compute the residual difference between all 4 quadrants
4. iteratively move the centre position and repeat 2. and 3. until all 4 $I(Q)$ curves lie on top of each other

## Initial guess: center-of-mass calculation

Computing the center-of-mass is straightforward using the vector `position` coordinate.

In [ ]:
com = bcf.center_of_mass(sample)

# We compute the shift between the incident beam direction and the center-of-mass
incident_beam = sample.transform_coords('incident_beam', graph=graph).coords[
    'incident_beam'
]
n_beam = incident_beam / sc.norm(incident_beam)
com_shift = com - sc.dot(com, n_beam) * n_beam
com_shift

We can now plot the center-of-mass on the same image as before:

In [ ]:
xc = com_shift.fields.x
yc = com_shift.fields.y
p = image.plot(norm='log', aspect='equal')
p.ax.plot(xc.value, yc.value, 'o', color='red', ms=5)
p

## Making 4 quadrants

We divide the panel into 4 quadrants.

In [ ]:
p = image.plot(norm='log', aspect='equal')
p.ax.plot(xc.value, yc.value, 'o', color='red', ms=5)
p.ax.axvline(xc.value, color='cyan')
p.ax.axhline(yc.value, color='cyan')
dx = 0.25
p.ax.text(xc.value + dx, yc.value + dx, 'North-East', ha='center', va='center')
p.ax.text(xc.value - dx, yc.value + dx, 'North-West', ha='center', va='center')
p.ax.text(xc.value - dx, yc.value - dx, 'South-East', ha='center', va='center')
p.ax.text(xc.value + dx, yc.value - dx, 'South-West', ha='center', va='center')
p

## Converting to $Q$ inside each quadrant

We now perform a full[$^1$](#footnote1) $I(Q)$ reduction (see [here](../../instruments/loki/sans2d_to_I_of_Q.ipynb) for more details) inside each quadrant.
The reduction involves computing a normalizing term which, for the most part, does not depend on pixel positions.
We therefore compute this once, before starting iterations to refine the position of the center.

### First compute normalizing term to avoid loop over expensive compute

To compute the denominator, we need to preprocess the monitor data from the sample and direct runs.
This involved converting them to wavelength and removing any background noise from the signal.

In [ ]:
# Extract monitor data
sample_monitors = {
    'incident': sample.attrs["monitor2"].value,
    'transmission': sample.attrs["monitor4"].value,
}
direct_monitors = {
    'incident': direct.attrs["monitor2"].value,
    'transmission': direct.attrs["monitor4"].value,
}
# Define the range where monitor data is considered not to be noise
non_background_range = sc.array(
    dims=['wavelength'], values=[0.7, 17.1], unit='angstrom'
)
# Pre-process monitor data
sample_monitors = sans.i_of_q.preprocess_monitor_data(
    sample_monitors,
    non_background_range=non_background_range,
    wavelength_bins=wavelength_bins,
)
direct_monitors = sans.i_of_q.preprocess_monitor_data(
    direct_monitors,
    non_background_range=non_background_range,
    wavelength_bins=wavelength_bins,
)

We then feed this, along with the sample run data (needed to include the detector pixel solid angles),
to a function which will compute the normalization term.

In [ ]:
norm = sans.normalization.iofq_denominator(
    data=sample,
    data_transmission_monitor=sc.values(sample_monitors['transmission']),
    direct_incident_monitor=sc.values(direct_monitors['incident']),
    direct_transmission_monitor=sc.values(direct_monitors['transmission']),
)

norm

### Compute $I(Q)$ inside the 4 quadrants

We begin by defining several parameters which are required to compute $I(Q)$.

In [ ]:
# Define 4 phi bins
pi = sc.constants.pi.value
phi_bins = sc.linspace('phi', -pi, pi, 5, unit='rad')

# Name the quadrants
quadrants = ['south-west', 'south-east', 'north-east', 'north-west']

# Define a wavelength range to use
wavelength_range = sc.concat(
    [wavelength_bins.min(), wavelength_bins.max()], dim='wavelength'
)

We now use a function which will apply the center offset to the pixel coordinates,
and compute $I(Q)$ inside each quadrant.

In [ ]:
grouped = sans.beam_center_finder.iofq_in_quadrants(
    xy=[com_shift.fields.x.value, com_shift.fields.y.value],
    sample=sample,
    norm=norm,
    graph=graph,
    q_bins=q_bins,
    gravity=gravity,
    wavelength_range=wavelength_range,
)

We can now plot on the same figure all 4 $I(Q)$ curves for each quadrant.

In [ ]:
pp.plot(grouped, norm='log')

As we can see, the overlap between the curves from the 4 quadrants is not satisfactory.
We will now use an iterative procedure to improve our initial guess, until a good overlap between the curves is found.

For this, we first define a cost function `sans.beam_center_finder.cost`, which gives us an idea of how good the overlap is:

$$
\text{cost} = \frac{\sum_{Q}\sum_{i=1}^{i=4} \overline{I}(Q)\left(I(Q)_{i} - \overline{I}(Q)\right)^2}{\sum_{Q}\overline{I}(Q)} ~,
$$

where $\overline{I}(Q)$ is the mean intensity of the 4 quadrants (represented by $i$) as a function of $Q$.
This is basically a weighted mean of the square of the differences between the $I(Q)$ curves in the 4 quadrants with respect to $\overline{I}(Q)$,
and where the weights are $\overline{I}(Q)$.

Next, we iteratively minimize the computed cost
(this is using Scipy's `optimize.minimize` utility internally;
see [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) for more details).

In [ ]:
# The minimizer works best if given bounds, which are the bounds of our detector panel
x = sample.coords['position'].fields.x
y = sample.coords['position'].fields.y

res = bcf.minimize(
    sans.beam_center_finder.cost,
    x0=[com_shift.fields.x.value, com_shift.fields.y.value],
    args=(sample, norm, graph, q_bins, gravity, wavelength_range),
    bounds=[(x.min().value, x.max().value), (y.min().value, y.max().value)],
)
res

Once the iterations completed, the returned object contains the best estimate for the beam center:

In [ ]:
res.x

We can now feed this value again into our `iofq_in_quadrants` function, to inspect the $Q$ intensity in all 4 quadrants:

In [ ]:
grouped = sans.beam_center_finder.iofq_in_quadrants(
    xy=[res.x[0], res.x[1]],
    sample=sample,
    norm=norm,
    graph=graph,
    q_bins=q_bins,
    gravity=gravity,
    wavelength_range=wavelength_range,
)

pp.plot(grouped, norm='log')

The overlap between the curves is excellent, allowing us to safely perform an azimuthal summation of the counts around the beam center.

As a consistency check, we plot the refined beam center position onto the detector panel image:

In [ ]:
p = sample.sum('tof').copy().hist(y=120, x=128).plot(norm='log', aspect='equal')
p.ax.plot(res.x[0], res.x[1], 'o', color='red', ms=5)
p

## Footnotes

<div id='footnote1'></div>

1. In the full $I(Q)$ reduction, there is a term $D(\lambda)$ in the normalization called the "direct beam" which gives the efficiency of the detectors as a function of wavelength.
Because finding the beam center is required to compute the direct beam in the first place,
we do not include this term in the computation of $I(Q)$ for finding the beam center.
This changes the shape of the $I(Q)$ curve, but since it changes it in the same manner for all $\phi$ angles,
this does not affect the results for finding the beam center.